In [1]:
from allennlp.models.archival import load_archive
from allennlp.data.dataset import Batch
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.predictors import Predictor
import csv, sys
from tqdm import tqdm
sys.path.append('/home/tony/QAMetric/Discriminative-Filtering/files')
sys.path.append('../merge_predictions/')

from model import Bert
from dataset_reader import ParaphraseDatasetReader
from merge_utils import *

In [39]:
archive = load_archive('../backtranslation/cosmosqa_discriminator//model.tar.gz')
model = archive.model.to(0).eval()
dataset_reader_params = archive.config.pop('dataset_reader')
dataset_reader_name = dataset_reader_params.pop('type')
reader = DatasetReader.by_name(dataset_reader_name).from_params(dataset_reader_params)

predictor = Predictor(model, reader)
with open('../backtranslation/cosmosqa/dev.csv_answers.backtranslations', 'r', encoding='utf8', errors='ignore') as fp:
    for i, line in enumerate(csv.reader(fp)):
        if i > 100:
            break
        context = reader.clean_string(line[1])
        question = reader.clean_string(line[2])
        answer = reader.clean_string(line[3])
        backtranslations = [reader.clean_string(eval(c)[0]) for c in line[5:]]
        
        print(question)
        instance = reader.text_to_instance(context + ' ' + question, answer)
        output_dict = predictor.predict_instance(instance)
        print(answer, output_dict['class_probabilities'])
        scores = []
        for bt in backtranslations:
            if bert_tokenization_length(context, question, answer, bt) + 4 > 512:
                continue
            
            if bt.lower() == answer.lower():
                continue
            instance = reader.text_to_instance(context + ' ' + question, bt)
            output_dict = predictor.predict_instance(instance)
            scores.append((bt, output_dict['class_probabilities']))
        
        scores = sorted(scores, key=lambda x: x[1], reverse=True)
        for bt, score in scores:
            print(bt, score)
            
        print('\n')

Why is this person asking about divorce ?
He wants to get married to a different person . 0.4316895306110382
she wants to marry another person . 0.48508602380752563
he wants to marry a very different person . 0.4345947206020355
he wants to get married to another man . 0.4338518977165222
he wants to marry a completely different person . 0.42969557642936707
he wants to get married to another person . 0.426552951335907
he wants to get married to another person . 0.426552951335907
he wants to marry another person . 0.4260266423225403
he wants to be married to another person . 0.41841408610343933
he wants to marry someone else . 0.4057570695877075
he wants to marry somebody else . 0.3936362862586975
he wants to marry a friend of the man . 0.276814341545105


Why is he worried about getting married ?
He was married before and she might come back one day . 0.5332651734352112
he had been married before and maybe one day again . 0.5281452536582947
he was married before and she came back perhaps

it makes no difference to Baby E , which is all about him around . 0.34155598282814026
it makes no difference to Baby E , which is about him around him . 0.32111579179763794
no matter what Baby E What happens around him , no matter . 0.3116944432258606
no matter Baby E , what happens around him . 0.27180737257003784
no matter Baby E , what happens around him . 0.27180737257003784
it makes no difference to baby E , which happens around him . 0.222372367978096
no matter what Baby E What's going on around him . 0.11033739894628525
no matter what Baby E What's happening around him . 0.102611243724823
no matter what Baby E What's going on around him , no matter . 0.08317247033119202
it doesn't matter Baby E , what happens around him . 0.024117987602949142
it doesn't matter Baby E , what happens around him . 0.024117987602949142


Why is it a good idea to shop for insurance regularly ?
It helps make sure that you are getting the best deal possible . 0.560990035533905
it helps make sure you g

they would n't write about it 0.6206305623054504
they wo n't write a story about it 0.5785237550735474
they wo n't write about it 0.5650705695152283
they wo n't write history about it 0.5419116020202637
they wouldn't have written 0.05209272727370262
they wouldn't have written a story 0.047979552298784256
they wouldn't have written an article 0.039266981184482574
they wouldn't write about it 0.030815137550234795
you would n't write a story about it 0.030244452878832817
you would not write a story about it 0.0284982118755579
you would not write a story about it 0.0284982118755579


Why can the writer foresee pulling their hair out ?
Because they are frustrated . 0.606343150138855
because they are disappointed . 0.5714834332466125
they are frustrated . 0.5536198616027832
they are frustrated at this . 0.5053696632385254
so they are disappointed . 0.457676500082016
there they are frustrated . 0.4165516197681427
so they're disappointed . 0.02862248755991459
because they're frustrated . 0.027

I wanted to take a break from my family . 0.46622586250305176
I wanted to take a break from my family . 0.46622586250305176
I wanted to take a break from my family . 0.46622586250305176
I wanted a break away from my family . 0.4542878568172455
I wanted to escape my family . 0.4453827738761902
I wanted to break my family's life . 0.3937683701515198
I wanted to break my family . 0.29545649886131287
I wanted to break my family . 0.29545649886131287
I wanted to take a break from the family . 0.2104646861553192
I wanted to take a break from the family . 0.2104646861553192


What may be your reason for going to Starbucks ?
Someone invited me there . 0.454600065946579
Someone has invited me there . 0.4957115948200226
someone invited me back there . 0.44998881220817566
somebody asked me out . 0.43389374017715454
somebody asked me out . 0.43389374017715454
somebody invited me there . 0.43264368176460266
someone invited me . 0.4313663840293884
someone asked me here . 0.4254959225654602
`` Someon

I wanted to look at some memories . 0.47529080510139465
I wanted to look at some memories . 0.47529080510139465
I wanted to see some memories . 0.4697839021682739
I wanted to see some memories . 0.4697839021682739
I wanted to look back at some memories . 0.46745765209198
I wanted to look back at some memories . 0.46745765209198
I wanted to look back at some memories . 0.46745765209198
I wanted to look back at some of the memories . 0.2675250470638275
I wanted to look back , for some memories . 0.18861210346221924
I wanted to look back , for some recollection . 0.1818467080593109


How do I feel about this person that is with me ?
I do n't really like her as much as I like the other girl . 0.49859490990638733
I really love her just like I love another girl . 0.32995644211769104
I really love her just like I like another girl . 0.32714730501174927
I really love her just like I love the other girl . 0.26117345690727234
I really love her just like I love the other girl . 0.2611734569072723

they are officers 0.5849784016609192
they are officers . 0.5688282251358032
they are officer 0.4782932996749878
they're officers . 0.029871948063373566
they're the officers . 0.029769327491521835
they're an officer . 0.027106154710054398
they're an officer 0.02687208540737629
you are an officer 0.02172190509736538


What branch of military is the narrator likely in ?
Air force 0.24957060813903809
aviation 0.5741968154907227
the Air Force 0.5122075080871582
air travel 0.4145486652851105
aviachou 0.08631505072116852
aviashou 0.06668172031641006


What has caused your blood pressure to soar the way it has ?
I was worried and concerned about the issues I was having with sorting out my travel itinerary 0.4964641034603119
I was worried and worried about my trip . 0.37965720891952515
I was concerned and concerned about the questions with which I sorted my travel schedule 0.36439502239227295
I was worried and concerned about the questions with which I sorted my travel schedule 0.35862565040588

because he has a job . 0.5559270977973938
because he has a job . 0.5559270977973938
because he needs to work there . 0.5394828915596008
because he has to work there . 0.5204000473022461
because he's out there and works . 0.07275141030550003
because he's there and works . 0.07248524576425552
because he's out there working . 0.06588614732027054
because he's busy . 0.06254326552152634
because he's running there . 0.0543418787419796
because he's working there . 0.050428565591573715
because he's got a job . 0.048183076083660126


Why are people concerned about me ?
I have been unemployed for a while now . 0.5838011503219604
I have been working for some time now . 0.6119470596313477
I have n't been working for some time now . 0.605342447757721
I have n't been working for a while now . 0.6030300259590149
I have been unemployed for some time now . 0.5894587635993958
I've been working for some time now . 0.025387730449438095
I've been unemployed for quite some time . 0.024213651195168495
I've b

because they have forgotten to bring them up . 0.6091174483299255
because they have forgotten to bring with me . 0.5433604717254639
because they forgot to bring them in . 0.5068517327308655
because they forgot to bring with me . 0.4456319510936737
because they forgot to take it . 0.43931400775909424
they forgot to bring it with them . 0.4348529875278473
they forgot to take them . 0.4336194694042206
there they forgot to bring it with them . 0.3512852191925049
because they forgot to take it with you . 0.18558701872825623
they forgot to take them with you . 0.10433939099311829


What may have been your reason for bring Nathan to the wedding ?
He could make it fun . 0.059024643152952194
he was able to make it fun . 0.39232638478279114
he can enjoy it . 0.06351203471422195
he could have fun . 0.059001196175813675
he could have fun . 0.059001196175813675
he could have made it fun . 0.05415891110897064
he could be fun to do . 0.05392104387283325
he could do it . 0.05236705765128136
he could b

because it will make their minds clearer . 0.6235336661338806
because it will make them minds stronger . 0.6007272005081177
because that will make their mind sharper . 0.44990670680999756
because that will make her brain sharper sharper . 0.41474610567092896
because that will make her brain sharper . 0.39680537581443787
because that will make their minds . 0.3651490807533264
because that will make your mind sharper . 0.33925122022628784
because that will make their mind . 0.3251834511756897
because that would make their minds clearer . 0.3036898374557495
because that would make them minds stronger . 0.292192280292511


What may have caused him to be scared ?
I was injured . 0.40803560614585876
I was getting injured . 0.547346830368042
I was getting injured . 0.547346830368042
I got injured . 0.44741761684417725
I got injured . 0.44741761684417725
I was hurt . 0.4308316707611084
I was wounded . 0.4126344621181488
I was wounded . 0.4126344621181488
I was wounded . 0.4126344621181488


Wh

he played a character that was not solely based on attractiveness . 0.49842414259910583
he played a character that was n't based solely on how to be attractive 0.4983726143836975
he played a character that was not based solely on what he attracted 0.49764496088027954
he played a character that was not based solely on what he was appealing 0.49683618545532227
he played a character that was not solely on attractiveness . 0.48669421672821045
he played a character that was not based solely on how to be attractive 0.4854140281677246
he played a character who was not alone on attractive 0.3422532379627228
he was playing a character who wasn't based solely on attraction 0.028017206117510796
he was playing a character who wasn't based solely on appeal . 0.026255276054143906
he played a character who wasn't based solely on attraction 0.024224117398262024
he played a character who wasn't based solely on appeal . 0.02334660477936268


How do I feel about Sid Lucero ?
I felt that Sid Lucero was a 